In [12]:


import pandas as pd
from huggingface_hub import hf_hub_download

REPO_ID = "phil329/OpenVid-1M-mapping"
dfs = []
for part in range(186):
    filename = f"video_mappings/OpenVid_part{part}.csv"
    file_path = hf_hub_download(
        repo_id=REPO_ID,
        filename=filename,
        repo_type="dataset",
        local_dir="."
    )
    df = pd.read_csv(file_path)
    dfs.append(df)

dataset = pd.concat(dfs, ignore_index=True)

print(dataset.head())


            zip_file                     video  \
0  OpenVid_part0.zip  celebv_--Jiv5iYqT8_0.mp4   
1  OpenVid_part0.zip  celebv_--Jiv5iYqT8_2.mp4   
2  OpenVid_part0.zip  celebv_--QCZKgJt6o_0.mp4   
3  OpenVid_part0.zip  celebv_--oCWVOBuvA_0.mp4   
4  OpenVid_part0.zip  celebv_--uyzf7X_0c_0.mp4   

                               video_path  
0  OpenVid_part1/celebv_--Jiv5iYqT8_0.mp4  
1  OpenVid_part1/celebv_--Jiv5iYqT8_2.mp4  
2  OpenVid_part1/celebv_--QCZKgJt6o_0.mp4  
3  OpenVid_part1/celebv_--oCWVOBuvA_0.mp4  
4  OpenVid_part1/celebv_--uyzf7X_0c_0.mp4  


In [13]:
from IPython.display import Video

Video("extracted_videos/celebv_--Jiv5iYqT8_0.mp4", embed=True)


In [14]:
# Versions I used that actually ran

# pip install "numpy==1.23.5" \
#            "tensorflow>=2.12,<3" \
#            "scipy<1.11" \
#            "h5py<4" \
#            "requests>=2.32.2" \
#            "gensim" \
#            "streamlit" \
#            "d2l"

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, ConvLSTM2D, Reshape, TimeDistributed
from tensorflow.keras.models import Model

sequence_length = 3  # frames encoded
height = 64          # frame height
width = 64           # frame width
channels = 3         # color channels rgb

input_shape = (sequence_length, height, width, channels)

inputs = Input(shape=input_shape)

# just looks at each frame separately, no temporal
x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'))(inputs)
x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same'))(x)

# x = Reshape((sequence_length, height, width, 64))(x)

# captures temporal
x = ConvLSTM2D(64, (3, 3), activation='relu', padding='same', return_sequences=True)(x)

# decoder
x = TimeDistributed(Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))(x)
x = TimeDistributed(Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))(x)
outputs = TimeDistributed(Conv2DTranspose(channels, (3, 3), activation='sigmoid', padding='same'))(x)

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 3, 64, 64, 3)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 3, 64, 64, 32)  │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 3, 64, 64, 64)  │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d (ConvLSTM2D)        │ (None, 3, 64, 64, 64)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 3, 64, 64, 64)  │        36,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 3, 64, 64, 32)  │        18,464 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 3, 64, 64, 3)   │           867 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 370,819 (1.41 MB)

 Trainable params: 370,819 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import os
import cv2
import numpy as np

# Path to the folder containing videos
video_folder = "extracted_videos"

# Function to preprocess videos
def preprocess_videos(video_folder, sequence_length, height, width, channels):
    video_sequences = []
    for video_file in os.listdir(video_folder):
        video_path = os.pat
        h.join(video_folder, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # Resize frame to the correct dimensions
            frame = cv2.resize(frame, (width, height))
            # Normalize pixel values to [0, 1]
            frame = frame / 255.0
            frames.append(frame)
        cap.release()
        # Convert frames to sequences of the correct length
        for i in range(len(frames) - sequence_length + 1):
            sequence = frames[i:i + sequence_length]
            video_sequences.append(sequence)
    return np.array(video_sequences)

# Preprocess the videos
video_data = preprocess_videos(video_folder, sequence_length, height, width, channels)

# Split the data into training and validation sets
split_index = int(0.8 * len(video_data))
train_data = video_data[:split_index]
val_data = video_data[split_index:]

# Reshape the training and validation data to match the input shape of the autoencoder
train_data = train_data.reshape((-1, sequence_length, height, width, channels))
val_data = val_data.reshape((-1, sequence_length, height, width, channels))

print("Train data shape:", train_data.shape)  # Expected: (num_samples, sequence_length, height, width, channels)
print("Validation data shape:", val_data.shape)


# Train the autoencoder
autoencoder.fit(
    train_data, train_data,
    validation_data=(val_data, val_data),
    epochs=10,
    batch_size=8
)

NameError: name 'sequence_length' is not defined

In [9]:
import os
import cv2
import numpy as np


def extract_video_sequences(video_path, sequence_length, height, width):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (width, height))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()

    sequences = []
    for i in range(len(frames) - sequence_length + 1):
        seq = frames[i:i+sequence_length]
        sequences.append(seq)
    return np.array(sequences)


In [7]:
def save_comparison_video(original, reconstructed, path="side_by_side.avi", fps=10):
    height, width = original.shape[2], original.shape[3]
    out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width * 2, height))

    for seq_idx in range(original.shape[0]):
        for t in range(original.shape[1]):
            orig_frame = (original[seq_idx, t] * 255).astype(np.uint8)
            rec_frame = (reconstructed[seq_idx, t] * 255).astype(np.uint8)
            combined = np.hstack((orig_frame, rec_frame))
            out.write(cv2.cvtColor(combined, cv2.COLOR_RGB2BGR))
    out.release()
    print(f"[✓] Side-by-side saved: {path}")


In [10]:
video_path = os.path.join(video_folder, os.listdir(video_folder)[0])
original_sequences = extract_video_sequences(video_path, sequence_length, height, width)
reconstructed_sequences = autoencoder.predict(original_sequences)

NameError: name 'sequence_length' is not defined

In [15]:
import os

# Save the reconstructed video only (optional)
def save_reconstructed_only(reconstructed, path="reconstructed.avi", fps=10):
    height, width = reconstructed.shape[2], reconstructed.shape[3]
    out = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    for seq in reconstructed:
        for frame in seq:
            frame = (frame * 255).astype(np.uint8)
            out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    out.release()
    print(f"[✓] Reconstructed-only video saved: {path}")

# Save compressed numpy version of the video (simulating model-side encoding)
np.savez_compressed("reconstructed_compressed.npz", data=reconstructed_sequences)


In [18]:
import cv2

def play_video(video_path, window_name="Video"):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow(window_name, frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):  # Press 'q' to quit
            break

    cap.release()
    cv2.destroyAllWindows()


In [17]:
def get_file_size(filepath):
    return os.path.getsize(filepath) / 1024

save_reconstructed_only(reconstructed_sequences)
save_comparison_video(original_sequences, reconstructed_sequences)

original_size = get_file_size(video_path)
reconstructed_size = get_file_size("reconstructed.avi")
side_by_side_size = get_file_size("side_by_side.avi")
compressed_size = get_file_size("reconstructed_compressed.npz")

print("\n📦 File Size Comparison:")
print(f"Original video:         {original_size:.2f} KB")
print(f"Reconstructed video:    {reconstructed_size:.2f} KB")
print(f"Side-by-side video:     {side_by_side_size:.2f} KB")
print(f"Compressed .npz array:  {compressed_size:.2f} KB")


[✓] Reconstructed-only video saved: reconstructed.avi
[✓] Side-by-side saved: side_by_side.avi

📦 File Size Comparison:
Original video:         263.68 KB
Reconstructed video:    83.66 KB
Side-by-side video:     143.10 KB
Compressed .npz array:  13631.30 KB


In [2]:
play_video("reconstructed.avi")
play_video("side_by_side.avi")

NameError: name 'play_video' is not defined